# ColumbiaX-04-Probabilistic-Matrix-Factorization

## Set Up Session

In [1]:
import numpy as np

## Data Generation

In [2]:
# Config:
d = 5
o_sigma2 = 0.1
h_lambda = 2

In [3]:
# Number of users & objects:
N_u = 6
N_v = 8

In [4]:
# Generate user & object
h_mu = np.zeros(d)
h_cov = 1.0 / h_lambda * np.eye(d)
U = np.random.multivariate_normal(mean=h_mu, cov=h_cov, size=N_u)
V = np.random.multivariate_normal(mean=h_mu, cov=h_cov, size=N_v)

In [5]:
# Number of observations:
N_o = np.random.randint(N_u*N_v)

In [6]:
# Generate observations:
index = np.unique(
    np.random.randint(N_u*N_v, size=N_o)
)
user_index, object_index = index / N_v, index % N_v

o_sigma = np.sqrt(o_sigma2)
rating = np.asarray(
    [
        np.random.normal(
            loc = np.sum(U[u]*V[o]),
            scale = o_sigma2
        ) for u, o in zip(user_index, object_index)
    ]
)

O = np.zeros((N_u, N_v))
for u, o, r in zip(user_index, object_index, rating):
    O[u, o] = r

In [7]:
# Generate dataset:
import pandas as pd

df_rating = pd.DataFrame(
    {
        'user_index': user_index+1, 
        'object_index': object_index+1, 
        'rating': rating
    }
)[['user_index', 'object_index', 'rating']]

df_rating.to_csv('ratings.csv', index=False, header=False)

## Probabilistic Matrix Factorization

$$
\mathcal{L} = -\sum_{(i,j)\in\Omega} \frac{1}{2\sigma^2}(M_{ij} - u_i^Tv_j)^2  - \sum_{i=1}^{N_u}\frac{\lambda}{2}\|u_i\|^2 - \sum_{j=1}^{N_v}\frac{\lambda}{2}\|v_j\|^2
$$

In [8]:
## Parse ratings:
def parse_ratings(filename):
    with open(filename) as f:
        lines = f.readlines()
    ratings = np.asarray(
        [[float(val) for val in line.split(",")] for line in lines]
    )
    return ratings
# 'ratings.csv'
X = parse_ratings('ratings.csv')

In [9]:
# Parse user & object index:
pmf_object_mapping, pmf_user_mapping = {}, {}
pmf_rating_mapping = {}
for pmf_user_index, pmf_object_index, pmf_rating in zip(
    X[:, 0].astype(np.int)-1, 
    X[:, 1].astype(np.int)-1,
    X[:, 2]
):
    # Update object mapping:
    object_list = pmf_object_mapping.get(pmf_user_index, [])
    object_list.append(pmf_object_index)
    pmf_object_mapping[pmf_user_index] = object_list
    # Update user mapping:
    user_list = pmf_user_mapping.get(pmf_object_index, [])
    user_list.append(pmf_user_index)
    pmf_user_mapping[pmf_object_index] = user_list
    # Update pmf_rating:
    pmf_rating_mapping[(pmf_user_index, pmf_object_index)] = pmf_rating

In [10]:
# Get the number of users & objects:
pmf_N_u, pmf_N_v = max(pmf_object_mapping.keys())+1, max(pmf_user_mapping.keys())+1

In [11]:
# Maximum number of iterations:
MAX_ITER = 50

In [12]:
# Initialize object matrix:
pmf_U = np.zeros((pmf_N_u, d))
pmf_h_mu = np.zeros(d)
pmf_h_cov = 1.0 / h_lambda * np.eye(d)
pmf_V = np.random.multivariate_normal(mean=pmf_h_mu, cov=pmf_h_cov, size = pmf_N_v)

In [13]:
# Probabilistic matrix factorization:
for epoch in xrange(MAX_ITER):
    # Update user matrix:
    for pmf_user_index in pmf_object_mapping.keys():
        pmf_object_list = pmf_object_mapping[pmf_user_index]
        pmf_U[pmf_user_index] = np.matmul(
            np.linalg.pinv(
                np.matmul(pmf_V[pmf_object_list].T, pmf_V[pmf_object_list]) + h_lambda * o_sigma2
            ),
            np.matmul(
                pmf_V[pmf_object_list].T,
                np.asarray([pmf_rating_mapping[(pmf_user_index, o)] for o in pmf_object_list])
            )
        )
    # Update user matrix:
    for pmf_object_index in pmf_user_mapping.keys():
        pmf_user_list = pmf_user_mapping[pmf_object_index]
        pmf_V[pmf_object_index] = np.matmul(
            np.linalg.pinv(
                np.matmul(pmf_U[pmf_user_list].T, pmf_U[pmf_user_list]) + h_lambda * o_sigma2
            ),
            np.matmul(
                pmf_U[pmf_user_list].T,
                np.asarray([pmf_rating_mapping[(u, pmf_object_index)] for u in pmf_user_list])
            )
        )

## Evaluate PMF Performance

In [15]:
# Prediction matrix:
pmf_O = np.matmul(
    pmf_U,
    pmf_V.T
)

In [17]:
# Evaluate PMF performance:
for pmf_user_index, pmf_object_index, pmf_rating in zip(
    X[:, 0].astype(np.int)-1, 
    X[:, 1].astype(np.int)-1,
    X[:, 2]
):
    print "[PMF Error]: {:.3f}".format(
        np.abs(pmf_rating-pmf_O[pmf_user_index, pmf_object_index])
    )

[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
[PMF Error]: 0.000
